# Config

In [14]:
%%writefile ./working/hyp.yaml

lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.30  # image rotation (+/- deg)
translate: 0.10  # image translation (+/- fraction)
scale: 0.10  # image scale (+/- gain)
shear: 2.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.5  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 0.2  # image mosaic (probability)
mixup: 0.5 # image mixup (probability)
copy_paste: 0.5  # segment copy-paste (probability)

Overwriting ./working/hyp.yaml


In [24]:
%%writefile ./working/yolov5x_0106.yaml


# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 1  # number of classes
depth_multiple: 1.33  # model depth multiple
width_multiple: 1.25  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]


Overwriting ./working/yolov5x_0106.yaml


In [25]:
image_size = 1280
batch_size = 16
epochs = 20
data = "../input/yolov5_fold0_nobbox003/data.yaml"
hyper_params = "./working/hyp.yaml"
model_config = "./working/yolov5l_0107.yaml"
weights = "./checkpoints/yolov5l.pt"
run_name = "yolov5l_e20_bs16_lr01_img1280_fold0_nobbox003_augv1"

# Train Model

In [26]:
!python train.py --img {image_size}\
--weights yolov5x.pt\
--cfg {model_config}\
--batch {batch_size}\
--epochs {epochs}\
--data {data}\
--hyp {hyper_params}\
--weights {weights}\
--name {run_name}\
--entity kk\
--project starfish\
--exist-ok

wandb: Currently logged in as: kk (use `wandb login --relogin` to force relogin)
train: weights=./checkpoints/yolov5l.pt, cfg=./working/yolov5l_0107.yaml, data=../input/yolov5_fold0_nobbox003/data.yaml, hyp=./working/hyp.yaml, epochs=20, batch_size=16, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=starfish, name=yolov5l_e20_bs16_lr01_img1280_fold0_nobbox003_augv1, exist_ok=True, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=kk, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 85afec1 torch 1.9.1+cu111 CUDA:0 (NVIDIA GeForce RTX 3090, 24268MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.00

      6/19     21.7G    0.0385   0.02059         0        25      1280: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        971       1341      0.709      0.406      0.466       0.23

     Epoch   gpu_mem       box       obj       cls    labels  img_size
      7/19     21.7G    0.0374   0.02042         0        30      1280: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        971       1341      0.721      0.436      0.505      0.248

     Epoch   gpu_mem       box       obj       cls    labels  img_size
      8/19     21.7G   0.03544   0.01959         0        37      1280: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        971       1341      0.745      0.535      0.571      0.284

     Epoch   gpu_mem       box       obj       cls    labels  img_size
      9/19     21.7G   0.03428   0.01